# Pauls Scrape Demo
------
This is a lightwieght demo to scrape data from the public facing NHLE list (Not because theres anything wrong with the offering, but more we'll not publicly upset anyone)

For this lets find a NHLE Record:

Go to: https://historicengland.org.uk/listing/the-list/
Search "Swindon"


Open the developer tools (F12) Navigate to the 





In [79]:
import requests
import csv
from bs4 import BeautifulSoup
 
# Making a GET request
request = requests.get('https://historicengland.org.uk/listing/the-list/list-entry/1355882?section=official-list-entry')

# check status code for response received
# success code - 200
result = "We've Got One" if request.status_code == 200 else "Who you going to call"
print(result)

We've Got One


Cool if you run the code below it will spit out the HTML of the page

In [12]:
soup = BeautifulSoup(request.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html building="" data-cms-template="NHLE List Entry" data-cms-type="Dummy" data-etl-image-server="https://historicengland.org.uk/etl/" data-fb-id="335322400195482" data-nhle-category="Listed" data-nhle-grade="II" data-server-name="HEPRODWEBIIS01" lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="index, follow" name="robots"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="DPR, Viewport-Width, Width" http-equiv="Accept-CH"/>
  <script src="/public/src/js/globals/globalVariables.js">
  </script>
  <script>
   globalVariables = setGlobalVariables();
        dataLayer = globalVariables.dataLayerArr;
        historicEngland = globalVariables.historicEnglandObj;
  </script>
  <script>
   window.dataLayer = window.dataLayer || [];
        function gtag() { dataLayer.push(arguments); }

        gtag('consent', 'default', {
            'ad_storage': 'denied',
            'analytics_storage': 'denied'
        });
  </script>


Cool lets pick out some other things

In [13]:
 # Getting the title tag
print(soup.title)
 
# Getting the name of the tag
print(soup.title.name)
 
# Getting the name of parent tag
print(soup.title.parent.name)

<title>24-27, Taunton Street, Swindon, SN1 5EE, Central Swindon South - 1355882 | Historic England</title>
title
head


Nice
Lets Pick apart the html 


In [14]:
# Finding by id
overview_Section = soup.find('section', id= 'official-list-entry')

# Dear Gatekeepers 
# This will also work:
#s = soup.find('section' ,attrs={"data-nhle-section": "Official Listing"})
# The DOM is a tree you can change it and it will slow me down! but you won't stop me
print(overview_Section)



<section data-nhle-section="Official Listing" id="official-list-entry">
<div class="nhle-entry__section">
<div class="container">
<div class="row my-0">
<div class="col-12">
<h2 class="mt-6">
                        Official list entry
                    </h2>
</div>
</div>
</div>
<div class="container">
<div class="row mt-0 position-relative">
<div class="col-lg-6">
<dl class="mt-5 mb-0">
<div>
<dt class="pt-0">
            Heritage Category:
        </dt>
<dd class="pt-0 fw-regular">
            Listed Building
        </dd>
</div>
<div>
<dt>Grade:</dt>
<dd class="fw-regular">
                II
            </dd>
</div>
<div>
<dt>List Entry Number:</dt>
<dd class="fw-regular text-nowrap">
                1355882
            </dd>
</div>
<div>
<dt>Date first listed:</dt>
<dd class="fw-regular text-nowrap">
                    17-Feb-1970
                </dd>
</div>
<div>
<dt>Statutory Address 1:</dt>
<dd class="fw-regular">
                    24-27, Taunton Street, Swindon, SN1 5EE

Now lets 

In [15]:
dl = overview_Section.div.contents[3].div.div.dl

print("Basic Details:")
for dl_div in list(dl.findAll("div")):
    print(dl_div.findNext("dt").get_text().replace(":", "").strip() + " -  " + dl_div.findNext("dd").get_text().strip() )

dl = overview_Section.div.contents[7].div.div.dl

print("Location Details:")
for dl_div in list(dl.findAll("div")):
    print(dl_div.findNext("dt").get_text().replace(":", "").strip() + " -  " + dl_div.findNext("dd").get_text().strip() )
dl = overview_Section.div.contents[7].div.div.contents[7]
for dl_div in list(dl.findAll("div")):
    print(dl_div.findNext("dt").get_text().replace(":", "").strip() + " -  " + dl_div.findNext("dd").get_text().strip() )

dl = overview_Section.div.contents[7].contents[5].div.dl

print("Legacy:")
for dl_div in list(dl.findAll("div")):
    print(dl_div.findNext("dt").get_text().replace(":", "").strip() + " -  " + dl_div.findNext("dd").get_text().strip() )

Basic Details:
Heritage Category -  Listed Building
Grade -  II
List Entry Number -  1355882
Date first listed -  17-Feb-1970
Statutory Address 1 -  24-27, Taunton Street, Swindon, SN1 5EE
Location Details:
Statutory Address -  24-27, Taunton Street, Swindon, SN1 5EE
District -  Swindon (Unitary Authority)
Parish -  Central Swindon South
National Grid Reference -  SU 14573 84702
Legacy:
Legacy System number -  318827
Legacy System -  LBS


Lets be honest thats enough.

So we can build 

In [16]:
dl = overview_Section.div.contents[3].div.div.dl
headerRow = []

for dl_div in list(dl.findAll("div")):
    headerRow.append(dl_div.findNext("dt").get_text().replace(":", "").strip())

dl = overview_Section.div.contents[7].div.div.dl

for dl_div in list(dl.findAll("div")):
    headerRow.append(dl_div.findNext("dt").get_text().replace(":", "").strip())

dl = overview_Section.div.contents[7].div.div.contents[7]
for dl_div in list(dl.findAll("div")):
   headerRow.append(dl_div.findNext("dt").get_text().replace(":", "").strip())

dl = overview_Section.div.contents[7].contents[5].div.dl

for dl_div in list(dl.findAll("div")):
    headerRow.append(dl_div.findNext("dt").get_text().replace(":", "").strip())

print(headerRow)

['Heritage Category', 'Grade', 'List Entry Number', 'Date first listed', 'Statutory Address 1', 'Statutory Address', 'District', 'Parish', 'National Grid Reference', 'Legacy System number', 'Legacy System']


In [76]:
import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

query = """select ?o where {
  ?s wdt:P1216 ?o
} LIMIT 10"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


wd_Result = get_results(endpoint_url, query)
lens = []
for result in wd_Result["results"]["bindings"]:
    lens.append(result['o']['value'])

In [91]:
table =[]
for ListNumber in lens:
    row = []
    forRequest = requests.get('https://historicengland.org.uk/listing/the-list/list-entry/' +ListNumber + '?section=official-list-entry')
    forSoup = BeautifulSoup(forRequest.content, 'html.parser')
    forOverview_Section = forSoup.find('section', id= 'official-list-entry')

    ix = 0
    for c in list(forOverview_Section.div.contents):
        ix = ix +1

    if(ix < 8):
        continue
    
    for c in list(forOverview_Section.div.contents[7].div.div.contents):
            ix = ix +1

    if(ix < 18):
        continue

    inner_dl = forOverview_Section.div.contents[3].div.div.dl
    for dl_div in list(inner_dl.findAll("div")):
        row.append(dl_div.findNext("dd").get_text().strip())

    inner_dl = forOverview_Section.div.contents[7].div.div.dl

    for dl_div in list(inner_dl.findAll("div")):
        row.append(dl_div.findNext("dd").get_text().replace(":", "").strip())
    
    inner_dl = forOverview_Section.div.contents[7].div.div.contents[7]
    for dl_div in list(inner_dl.findAll("div")):
        row.append(dl_div.findNext("dd").get_text().replace(":", "").strip())
   
    table.append(row)

print(table)


with open('OutPut/Sample.csv','w', newline='') as csvfile:
    write = csv.writer(csvfile)
    write.writerow(headerRow)
    write.writerows(table)

[['Battlefield', '1000007', '06-Jun-1995', 'CHERITON', 'Hampshire', 'Winchester (District Authority)', 'Bishops Sutton', 'Hampshire', 'Winchester (District Authority)', 'Bramdean and Hinton Ampner', 'Hampshire', 'Winchester (District Authority)', 'Cheriton', 'SOUTH DOWNS', 'SU 59775 29360'], ['Park and Garden', 'I', '1000111', '01-Oct-1987', 'Greater London Authority', 'Hounslow (London Borough)', 'Non Civil Parish', 'TQ 20881 77637'], ['Listed Building', 'II', '1028458', '22-Aug-1986', 'TEDDARDS BARN YARD ATTACHED', 'TEDDARDS BARN YARD ATTACHED', 'East Sussex', 'Wealden (District Authority)', 'Willingdon and Jevington', 'SOUTH DOWNS', 'TQ 56181 02940'], ['Listed Building', 'II', '1032346', '24-Jun-1988', 'BARN IMMEDIATELY WEST OF BEDFIELD LODGE, LONG GREEN', 'BARN IMMEDIATELY WEST OF BEDFIELD LODGE, LONG GREEN', 'Suffolk', 'Mid Suffolk (District Authority)', 'Bedfield', 'TM 21556 67564'], ['Listed Building', 'I', '1057497', '22-May-1969', 'Combe Sydenham', 'Combe Sydenham', 'Somerset'